In [ ]:
pip install os opencv-contrib-python keras tensorflow numpy

In [1]:
import os
import cv2

In [2]:
def make_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        return None
    else:
        pass

In [3]:
# DataSet Building
cap = cv2.VideoCapture(0)

i=0
image_Count=0

while i < 7:

    ret, frame = cap.read()
    frame = cv2.flip(frame,1)

    #region of intereset
    roi = frame[100:400, 320:600]
    cv2.imshow('roi',roi)
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    roi = cv2.resize(roi , (28, 28) ,interpolation = cv2.INTER_AREA)

    cv2.imshow('roi sacled and gray ',roi)
    copy = frame.copy()
    cv2.rectangle(copy,(320, 100) ,(620, 400) ,(255,0,0), 5)

    if i == 0:
        image_Count = 0
        cv2.putText(copy, " Hit Enter to Record when Ready " ,(100,100) , cv2.FONT_HERSHEY_COMPLEX, 1,(0,255,0), 1)

    if i == 1:
        image_Count += 1
        cv2.putText(copy, " Recording 1st Gesture -Train ", (100, 100), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
        cv2.putText(copy,str(image_Count) , (400, 400), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
        gesture_one = './handgestures/train/0/'
        make_dir(gesture_one)
        cv2.imwrite(gesture_one + str(image_Count) + ".jpg" ,roi)

    if i == 2:
        image_Count += 1
        cv2.putText(copy, " Recording 1st Gesture -Train ", (100, 100), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
        cv2.putText(copy, str(image_Count), (400, 400), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
        gesture_one = './handgestures/test/0/'
        make_dir(gesture_one)
        cv2.imwrite(gesture_one + str(image_Count) + ".jpg", roi)

    if i == 3:
        cv2.putText(copy, " Hit Enter to Record when Ready to recond 2nd gesture " ,(100,100) , cv2.FONT_HERSHEY_COMPLEX, 1,(0,255,0), 1)

    if i == 4:
        image_Count += 1
        cv2.putText(copy, " Recording 2nd Gesture -Train ", (100, 100), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
        cv2.putText(copy,str(image_Count) , (400, 400), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
        gesture_two = './handgestures/train/1/'
        make_dir(gesture_two)
        cv2.imwrite(gesture_two + str(image_Count) + ".jpg" ,roi)

    if i == 5:
        image_Count += 1
        cv2.putText(copy, " Recording 2nd Gesture -Train ", (100, 100), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
        cv2.putText(copy, str(image_Count), (400, 400), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
        gesture_two = './handgestures/test/1/'
        make_dir(gesture_two)
        cv2.imwrite(gesture_two + str(image_Count) + ".jpg", roi)

    if i == 6:
        cv2.putText(copy, " Hit Enter to Exit ", (100, 100),cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
    cv2.imshow('frame', copy)

    if cv2.waitKey(1) == 13 : #13 is Enter Key
        image_Count = 0
        i+=1

cap.release()
cv2.destroyAllWindows()

In [4]:
##DATA AUGMENTATION

In [5]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.models import load_model
import cv2

In [10]:

num_classes = 2
img_rows,img_cols = 28,28
batch_size = 32

train_data_dir ='./handgestures/train'
validation_data_dir ='./handgestures/test'
# data augmentation
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size =(img_rows,img_cols),
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size =(img_rows,img_cols),
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='binary')

Found 1370 images belonging to 2 classes.
Found 478 images belonging to 2 classes.


In [11]:

#Model Creation
model = Sequential()
model.add(Conv2D(64, kernel_size=(3,3), activation= 'relu', input_shape=(28,28,1) ))
model.add(MaxPooling2D(pool_size=(2,2) ))

model.add(Conv2D(64, kernel_size=(3,3), activation= 'relu' ))
model.add(MaxPooling2D(pool_size=(2,2) ))

model.add(Conv2D(64, kernel_size=(3,3), activation= 'relu' ))
model.add(MaxPooling2D(pool_size=(2,2) ))

model.add(Flatten())
model.add(Dense(128, activation= 'relu'))
model.add(Dropout(0.20))
#1<->num_Classes
model.add(Dense(1, activation= 'sigmoid'))

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 1, 1, 64)         0

In [12]:

#Training the model
model.compile(loss ='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

nb_train_samples = 1370
nb_validation_samples = 478
epochs = 10

history = model.fit(
    train_generator ,
    steps_per_epoch= nb_train_samples // batch_size,
    epochs=epochs,
    validation_data= validation_generator,
    validation_steps =nb_validation_samples // batch_size
)


Epoch 1/10
42/42 [==============================] - 5s 78ms/step - loss: 0.6910 - accuracy: 0.5448 - val_loss: 0.6715 - val_accuracy: 0.5982
Epoch 2/10
42/42 [==============================] - 3s 70ms/step - loss: 0.6289 - accuracy: 0.6592 - val_loss: 0.5635 - val_accuracy: 0.6004
Epoch 3/10
42/42 [==============================] - 3s 69ms/step - loss: 0.4752 - accuracy: 0.7825 - val_loss: 0.4780 - val_accuracy: 0.6071
Epoch 4/10
42/42 [==============================] - 3s 70ms/step - loss: 0.3287 - accuracy: 0.8797 - val_loss: 0.1627 - val_accuracy: 0.9933
Epoch 5/10
42/42 [==============================] - 4s 84ms/step - loss: 0.2629 - accuracy: 0.9058 - val_loss: 0.6540 - val_accuracy: 0.6272
Epoch 6/10
42/42 [==============================] - 3s 78ms/step - loss: 0.1973 - accuracy: 0.9380 - val_loss: 0.0179 - val_accuracy: 1.0000
Epoch 7/10
42/42 [==============================] - 3s 76ms/step - loss: 0.1546 - accuracy: 0.9537 - val_loss: 0.1033 - val_accuracy: 0.9732
Epoch 8/10
42

In [13]:
#Saving the Model
model.save("My_Gesture_CNN.h5")

classifier = load_model('My_Gesture_CNN.h5')



In [15]:
#Testing the Model

cap = cv2.VideoCapture(0)

while True:

    ret, frame =cap.read()
    frame = cv2.flip(frame,1)

    #defining roi
    # region of intereset
    x = 100
    y = 800
    w = 320
    z = 920
    roi = frame[x:y, w:z]
    cv2.imshow('roi', roi)
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    roi = cv2.resize(roi, (28, 28), interpolation=cv2.INTER_AREA)
    cv2.imshow('roi scaled and gray ', roi)
    copy = frame.copy()
    cv2.rectangle(copy, (320, 100), (620, 400), (255, 0, 0), 5)
    
    roi = roi.reshape(1,28,28,1)
    roi = roi/255
    y_predict = (classifier.predict(roi) > 0.5).astype("int32")
    # y_predict = np.argmax(classifier.predict(roi), axis=1)
    #classifier.predict_classes(roi,1, verbose = 0)
    result = str(y_predict[0][0])
    print("Result is : "+result)
    cv2.putText(copy,'Sign:'+str(result),(300,400),cv2.FONT_HERSHEY_COMPLEX, 2, (0, 255, 0), 2)
    cv2.imshow('frame',copy)

    if cv2.waitKey(1) == 13:  # 13 is Enter Key
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 36ms/step
Result is : 1
1/1 [==============================] - 0s 35ms/step
Result is : 1
1/1 [==============================] - 0s 43ms/step
Result is : 1
1/1 [==============================] - 0s 52ms/step
Result is : 1
1/1 [==============================] - 0s 52ms/step
Result is : 1
1/1 [==============================] - 0s 44ms/step
Result is : 1
1/1 [==============================] - 0s 32ms/step
Result is : 1
1/1 [==============================] - 0s 32ms/step
Result is : 1
1/1 [==============================] - 0s 37ms/step
Result is : 1
1/1 [==============================] - 0s 39ms/step
Result is : 1
1/1 [==============================] - 0s 35ms/step
Result is : 1
1/1 [==============================] - 0s 61ms/step
Result is : 1
1/1 [==============================] - 0s 49ms/step
Result is : 1
1/1 [==============================] - 0s 38ms/step
Result is : 1
1/1 [==============================] - 0s 36ms/step
Result is : 1
1/1 [=====

1/1 [==============================] - 0s 34ms/step
Result is : 1
1/1 [==============================] - 0s 30ms/step
Result is : 1
1/1 [==============================] - 0s 38ms/step
Result is : 1
1/1 [==============================] - 0s 32ms/step
Result is : 1
1/1 [==============================] - 0s 43ms/step
Result is : 1
1/1 [==============================] - 0s 41ms/step
Result is : 1
1/1 [==============================] - 0s 31ms/step
Result is : 1
1/1 [==============================] - 0s 33ms/step
Result is : 1
1/1 [==============================] - 0s 35ms/step
Result is : 1
1/1 [==============================] - 0s 32ms/step
Result is : 1
1/1 [==============================] - 0s 46ms/step
Result is : 1
1/1 [==============================] - 0s 39ms/step
Result is : 1
1/1 [==============================] - 0s 45ms/step
Result is : 1
1/1 [==============================] - 0s 41ms/step
Result is : 1
1/1 [==============================] - 0s 37ms/step
Result is : 1
1/1 [=====

1/1 [==============================] - 0s 43ms/step
Result is : 0
1/1 [==============================] - 0s 40ms/step
Result is : 0
1/1 [==============================] - 0s 39ms/step
Result is : 1
1/1 [==============================] - 0s 32ms/step
Result is : 1
1/1 [==============================] - 0s 34ms/step
Result is : 1
1/1 [==============================] - 0s 38ms/step
Result is : 1
1/1 [==============================] - 0s 32ms/step
Result is : 1
1/1 [==============================] - 0s 37ms/step
Result is : 1
1/1 [==============================] - 0s 36ms/step
Result is : 1
1/1 [==============================] - 0s 54ms/step
Result is : 1
1/1 [==============================] - 0s 43ms/step
Result is : 1
1/1 [==============================] - 0s 40ms/step
Result is : 0
1/1 [==============================] - 0s 33ms/step
Result is : 0
1/1 [==============================] - 0s 33ms/step
Result is : 0
1/1 [==============================] - 0s 35ms/step
Result is : 0
1/1 [=====

1/1 [==============================] - 0s 45ms/step
Result is : 0
1/1 [==============================] - 0s 46ms/step
Result is : 1
1/1 [==============================] - 0s 48ms/step
Result is : 1
1/1 [==============================] - 0s 44ms/step
Result is : 1
1/1 [==============================] - 0s 40ms/step
Result is : 1
1/1 [==============================] - 0s 40ms/step
Result is : 1
1/1 [==============================] - 0s 44ms/step
Result is : 1
1/1 [==============================] - 0s 44ms/step
Result is : 1
1/1 [==============================] - 0s 43ms/step
Result is : 1
1/1 [==============================] - 0s 51ms/step
Result is : 0
1/1 [==============================] - 0s 50ms/step
Result is : 0
1/1 [==============================] - 0s 36ms/step
Result is : 0
1/1 [==============================] - 0s 44ms/step
Result is : 0
1/1 [==============================] - 0s 45ms/step
Result is : 0
1/1 [==============================] - 0s 46ms/step
Result is : 0
1/1 [=====